In [1]:
import urllib
from bs4 import BeautifulSoup

In [57]:
url = "https://ethiopianspices.com/collections/all"
html = urllib.urlopen(url).read()
soup = BeautifulSoup(html,'lxml')

In [58]:
text = soup.get_text()

In [59]:
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

In [60]:
from lxml import html
import requests

In [61]:
page = requests.get('https://ethiopianspices.com/collections/all')
tree = html.fromstring(page.content)

In [83]:
## product=<div class="h4 grid-view-item__title product-card__title" aria-hidden="true">Berbere</div>
##price:  <span class="price-item price-item--sale" data-sale-price="">$9.50</span>
#This will create a list of products:
product = tree.xpath('//div[@class="h4 grid-view-item__title product-card__title"]/text()')
#This will create a list of prices
prices = tree.xpath('//span[@class="price-item price-item--sale"]/text()')


In [84]:
#strip new line
_prices=[price.strip() for price in prices]
_prices[0]

'$9.75'

In [85]:
print 'Product: %s' %product[0]
print 'Price: %s' %_prices[0]

Product: Alicha Kimem
Price: $9.75


In [86]:
#convert price to float type
prices=[float(price.split('$')[1]) for price in _prices]
prices[0]

9.75

In [87]:
#create a dataframe
import pandas as pd
df_price= pd.DataFrame({'spice':product,
                      'price':prices,
                       'unit size': '8oz'}, columns=['spice','price', 'unit size'])

In [88]:
#change engera Unite Size
df_price.loc[7,'unit size'] = '10pcs'

In [89]:
#SAVE TO EXCELL 
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Spice_List_Price_US.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df_price.to_excel(writer, sheet_name='EthiopianSpices.com')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [90]:
#Add new sheet with source
#import excell sheet
from openpyxl import load_workbook
path = u"/Users/bruktawitabebe/Desktop/Spice/Spice_List_Price_US.xlsx"
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book


In [91]:
#write new df_source to excell file
_dict= {'name': 'Ethio Imports LLC',
                       'source':'ethiopianspices.com',
                       'email':'ethiospiceimports@gmail.com',
                       'phone': '502-459-6301/502-243-5472',
                       'delivery_time_days': '3-7'}
source = pd.DataFrame({k: [v] for k, v in _dict.items()},columns=['name','source','email','phone','delivery_time_days'])

In [92]:
# Convert the dataframe to an XlsxWriter Excel object.
source.to_excel(writer, sheet_name='Source')
writer.save()

In [93]:
#read source sheet
pd.read_excel('Spice_List_Price_US.xlsx',sheetname='Source')

,name,source,email,phone,delivery_time_days
0,Ethio Imports LLC,ethiopianspices.com,ethiospiceimports@gmail.com,502-459-6301/502-243-5472,3-7


In [94]:
pd.read_excel('Spice_List_Price_US.xlsx',sheetname='EthiopianSpices.com').head()

,spice,price,unit size
0,Alicha Kimem,9.75,8oz
1,Berbere,9.50,8oz
2,Berbere Awaze,10.50,8oz
3,Beso Bela,11.50,8oz
4,Coffee Beans,7.50,8oz


In [95]:
#2nd data source read html
page = requests.get('http://www.goorsha.net/essential-ethiopian-spices/')
tree = html.fromstring(page.content)
#This will create a list of products:
product = tree.xpath('//div[@class="h4 grid-view-item__title product-card__title"]/text()')
#This will create a list of prices
prices = tree.xpath('//span[@class="price-item price-item--sale"]/text()')

In [96]:
import lxml.html as lh
url='http://www.goorsha.net/essential-ethiopian-spices/'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [97]:
[len(T) for T in tr_elements[:12]]

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]

In [98]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print '%d:"%s"'%(i,name)
    col.append((name,[]))

1:"INGREDIENT"
2:"AMHARIC NAME"
3:"LATIN NAME"
4:"ORIGIN"


In [99]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=4:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1


In [100]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [101]:
df.head(2)

,AMHARIC NAME,INGREDIENT,LATIN NAME,ORIGIN
0,Netch Azmud,"Bishop’s Weed, Ajowan",Trachyspermum ammi,Asia
1,Mit’mita,"African birdseye pepper, periperi",Capsicum frutescens (cultivar variet,South America


In [102]:
# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='EssentialIngredients')
writer.save()
#read source sheet
pd.read_excel('Spice_List_Price_US.xlsx',sheetname='EssentialIngredients').head()

,AMHARIC NAME,INGREDIENT,LATIN NAME,ORIGIN
0,Netch Azmud,"Bishop’s Weed, Ajowan",Trachyspermum ammi,Asia
1,Mit’mita,"African birdseye pepper, periperi",Capsicum frutescens (cultivar variet,South America
2,K’undo Berebere,Black Pepper,Piper nigrum,Indian Subcontinent
3,Kemun,Cumin,Cuminum cyminum,Mediterranean
4,Kewrerima,"False Cardamom, Grain of Paradise",Aframomum melegueta,West Africa


In [139]:
#3rd data source read html
page = requests.get('https://www.brundo.com/collections/spices?sort_by=title-ascending')
tree = html.fromstring(page.content)
#This will create a list of products:
product = tree.xpath('//span[@class="title"]/text()')
#This will create a list of prices
prices = tree.xpath('//span[@class="money"]/text()')

In [140]:
#extract other spices from different pages
page = requests.get('https://www.brundo.com/collections/chilli-pepper')
tree = html.fromstring(page.content)
#Append to product list 
[product.append(aprod) for aprod in tree.xpath('//span[@class="title"]/text()')]
#Append to list of prices
[prices.append(aprice) for aprice in tree.xpath('//span[@class="money"]/text()')]

[None, None, None, None, None, None, None, None]

In [141]:
tree.xpath('//span[@class="money"]/text()')

['$ 5.95',
 '$ 6.95',
 '$ 4.95',
 '$ 4.95',
 '$ 6.95',
 '$ 6.95',
 '$ 5.95',
 '$ 5.95']

In [142]:
product

[u'Abish | \u12a0\u1265\u1238 - Fenugreek',
 u'Bessobela | \u1260\u1236\u1265\u120b - Ethiopian Sacred Basil',
 u'Dimbilal | \u12f5\u1295\u1265\u120b\u120d',
 'Essential Spice Blends (5 M)',
 u'Ird | \u12a5\u122d\u12f5 - Turmeric',
 u'Kosseret | \u12ae\u1230\u1228\u1275 - Butter Clarifying Herb',
 u'Mekelesha | \u1218\u12a8\u1208\u123b - Finishing Spice Mix',
 u'Netch Azmud | \u1290\u132d \u12a0\u12dd\u1219\u12f5 - White Cumin Seed',
 u'Senafitch | \u1230\u1293\u134d\u127d - Mustard Seed',
 u'Tikur Azmud | \u1325\u1241\u122d \u12a0\u12dd\u1219\u12f5 - Black Cumin',
 u'Tossegen | \u1326\u1235\u129d - Ethiopian Thyme',
 u'Berbere | \u1260\u122d\u1260\u122c - Hot Chili Pepper Blend',
 u'Mitmita | \u121a\u1325\u121a\u1323 - Hot Chili  Blend',
 u'Awaze |  \u12a0\u12cb\u12dc - Chili Pepper Paste',
 u'Afrenje | \u12a0\u134d\u122d\u1295\u1305 - Pepper Seed']

In [143]:
product[3].strip()

'Essential Spice Blends (5 M)'

In [144]:
#extract amharic and english name
product_name=[]
english_name=[]
for prod in product:
    if '|' in prod:
        product_name.append(prod.split('|')[0])
    else:
        product_name.append(prod.strip())
    if '-' in prod:
        english_name.append(prod.split('-')[-1]) 
    else:
        english_name.append('None')
#extract float number from price
price=[float(aprice.split()[1]) for aprice in prices]
#duplicate prices, take every other price
prices=price[0::2]
print 'Spice is: %s'% product_name[0]
print 'Price is: %s' % price[0]

Spice is: Abish 
Price is: 6.95


In [145]:
#create df
df=pd.DataFrame({'spice': product_name,
               'english_name': english_name,
               'price': prices}, columns=['spice','english_name', 'price'])
df.head(2)

,spice,english_name,price
0,Abish,Fenugreek,6.95
1,Bessobela,Ethiopian Sacred Basil,7.95


In [146]:
df['spice']

0                           Abish 
1                       Bessobela 
2                        Dimbilal 
3     Essential Spice Blends (5 M)
4                             Ird 
5                        Kosseret 
6                       Mekelesha 
7                     Netch Azmud 
8                       Senafitch 
9                     Tikur Azmud 
10                       Tossegen 
11                        Berbere 
12                        Mitmita 
13                          Awaze 
14                        Afrenje 
Name: spice, dtype: object

In [147]:
#create unit size column
df['unit size'] = ['8oz','4oz','2oz','10oz','8oz','2oz','2oz','None','2oz','2oz','2oz','4oz','2oz','4oz','8oz']

In [148]:
#save df as new sheet
df.to_excel(writer, sheet_name='brundo.com')
writer.save()

In [149]:
#add new source code for df
source_df= pd.read_excel('Spice_List_Price_US.xlsx',sheetname='Source')

In [150]:
_dict= {'name': 'Brundo Market',
                       'source':'brundo.com',
                       'email':'info@brundo.com',
                       'phone': '(510) 601-7999',
                       'delivery_time_days': 'none'}
#source = pd.DataFrame({k: [v] for k, v in _dict.items()},columns=['name','source','email','phone','delivery_time_days'])

In [151]:
sourcedf=source_df.append(_dict, ignore_index=True)

In [152]:
#save updated source 
sourcedf.to_excel(writer, sheet_name='Source')
writer.save()

In [153]:
sourcedf

,name,source,email,phone,delivery_time_days
0,Ethio Imports LLC,ethiopianspices.com,ethiospiceimports@gmail.com,502-459-6301/502-243-5472,3-7
1,Brundo Market,brundo.com,info@brundo.com,(510) 601-7999,none
2,Brundo Market,brundo.com,info@brundo.com,(510) 601-7999,none


In [154]:
pd.read_excel('Spice_List_Price_US.xlsx',sheetname='brundo.com').head(2)

,spice,english_name,price,unit size
0,Abish,Fenugreek,6.95,8oz
1,Bessobela,Ethiopian Sacred Basil,7.95,4oz


In [155]:
df

,spice,english_name,price,unit size
0,Abish,Fenugreek,6.95,8oz
1,Bessobela,Ethiopian Sacred Basil,7.95,4oz
2,Dimbilal,None,4.95,2oz
3,Essential Spice Blends (5 M),None,24.99,10oz
4,Ird,Turmeric,14.95,8oz
5,Kosseret,Butter Clarifying Herb,5.95,2oz
6,Mekelesha,Finishing Spice Mix,7.95,2oz
7,Netch Azmud,White Cumin Seed,13.95,None
8,Senafitch,Mustard Seed,4.95,2oz
9,Tikur Azmud,Black Cumin,7.95,2oz


In [2]:
#import pdf 
import PyPDF2
PDFfilename = "example.pdf" #filename of your PDF/directory where your PDF is stored
pfr = PyPDF2.PdfFileReader(open(PDFfilename, "rb")) #PdfFileReader object


ImportError: No module named errors